# 파일명 변경 및 정크 데이터 삭제

In [42]:
import os
import re
from langdetect import detect
from mutagen.mp3 import MP3
from datetime import datetime  # datetime 클래스를 명시적으로 임포트

directory = './data/'  # MP3 파일들이 위치한 폴더 경로

# 파일을 생성 날짜에 따라 정렬
mp3_files = [f for f in os.listdir(directory) if f.endswith('.mp3')]
mp3_files.sort(key=lambda x: os.path.getctime(os.path.join(directory, x)))

def clean_filename(filename):
    """특수문자를 제외한 파일 이름을 반환합니다."""
    # 알파벳, 숫자, 공백, 하이픈, 언더스코어만 허용
    return re.sub(r'[^\w\s-]', '', filename)

# 파일 처리 및 이름 변경
for index, filename in enumerate(mp3_files, 1):
    try:
        file_path = os.path.join(directory, filename)
        audio = MP3(file_path)

        # 비영어 제목 확인
        if detect(filename) != 'en':
            os.remove(file_path)
            continue

        # 파일 길이 확인 (7분 미만 제거)
        if audio.info.length < 420:
            os.remove(file_path)
            continue

        # 파일 이름에서 특수문자 제거
        clean_name = clean_filename(filename)
        
        # 파일 생성 날짜를 '년-월-일' 형식으로 변환
        creation_time = datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y%m%d')

        # 확장자 유지
        extension = '.mp3'

        # 파일 이름 변경 (인덱스-생성 날짜-제목 형식)
        new_filename = f"{index}_{creation_time}_{clean_name}{extension}"
        new_file_path = os.path.join(directory, new_filename)
        os.rename(file_path, new_file_path)

    except Exception as e:
        print(f"Error processing file {filename}: {e}")


In [44]:
import os

for filename in os.listdir(directory):
    if filename.endswith('mp3'):  # 'mp3'로 끝나는 파일 이름 찾기
        correct_filename = filename[:-7] + '.mp3'  # 올바른 확장자 '.mp3'로 변경
        os.rename(os.path.join(directory, filename), os.path.join(directory, correct_filename))


In [41]:
import os

def remove_specific_part(filename, part_to_remove):
    """ 파일 이름에서 특정 부분을 제거하는 함수 """
    return filename.replace(part_to_remove, '', 1)  # 부분을 한 번만 제거

for filename in os.listdir(directory):
    if filename.endswith('.mp3'):
        # 제거하고자 하는 부분을 정의 (예: "-1-2023-11-15")
        parts = filename.split('_')
        part_to_remove = '_'.join(parts[0:2]) + '_'
        
        # 특정 부분을 제거
        new_filename = remove_specific_part(filename, part_to_remove)

        # 파일 이름 변경
        os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))


# 위스퍼를 통해 텍스트화

In [ ]:
import os
import natsort
from tqdm import tqdm
import time
import stable_whisper

directory_path = "./data"

# Load Whisper model
model = stable_whisper.load_faster_whisper('.', device="cuda", compute_type="float16")

# Walk through the directory
for root, dirs, files in os.walk(directory_path):
    files = natsort.natsorted(files)
    
    # Wrap file processing with tqdm
    with tqdm(total=len(files), desc="Processing files", unit="file") as pbar:
        for file in files:
            # Process only .mp3 files
            if file.endswith(".mp3"):
                video_file_path = os.path.join(root, file)
                video_filename = os.path.splitext(os.path.basename(video_file_path))[0]
                txt_filename = f'{video_filename}.txt'
                txt_file_path = f"./transcribe/{txt_filename}"

                # Check if the txt file already exists
                if os.path.exists(txt_file_path):
                    tqdm.write(f"Skipping {txt_filename} as it already exists.")
                else:
                    # Transcribe the video
                    result = model.transcribe_stable(video_file_path, vad=True, language="en", verbose=False)

                    # Save the result to a txt file
                    with open(txt_file_path, 'w', encoding='utf-8') as txt_file:
                        txt_file.write(result.text)

                    tqdm.write(f"Processing {file}")

                # Update the tqdm progress bar
                pbar.update(1)
